In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import RSI
from technicals.patterns import apply_patterns
from plotting import CandlePlot

In [2]:
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates

In [3]:
pair = "USDCHF"
NB_H1_CANDLES = 2000

In [4]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_raw = rates.get_rates_from_now()
df_raw.drop(999,inplace=True)
df_raw

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-11-21 11:00:00,0.95645,0.95938,0.95625,0.95920,5882,0,0
1,2022-11-21 12:00:00,0.95920,0.95977,0.95824,0.95830,5955,0,0
2,2022-11-21 13:00:00,0.95829,0.95876,0.95701,0.95796,4642,0,0
3,2022-11-21 14:00:00,0.95794,0.95905,0.95780,0.95829,4814,0,0
4,2022-11-21 15:00:00,0.95828,0.95889,0.95719,0.95785,5472,0,0
...,...,...,...,...,...,...,...,...
1995,2023-03-20 15:00:00,0.92737,0.92746,0.92422,0.92486,8854,0,0
1996,2023-03-20 16:00:00,0.92486,0.92599,0.92401,0.92518,7059,0,0
1997,2023-03-20 17:00:00,0.92518,0.92705,0.92505,0.92674,6573,0,0
1998,2023-03-20 18:00:00,0.92674,0.92686,0.92549,0.92610,6185,0,0


In [5]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD #= 0

0.0001

In [6]:
df_an = df_raw.copy()#df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [7]:
df_an = RSI(df_an)

In [8]:
df_an.tail()

,time,open,high,low,close,tick_volume,spread,real_volume,RSI_14
1994,2023-03-20 15:00:00,0.92737,0.92746,0.92422,0.92486,8854,0,0,40.835615
1995,2023-03-20 16:00:00,0.92486,0.92599,0.92401,0.92518,7059,0,0,42.430667
1996,2023-03-20 17:00:00,0.92518,0.92705,0.92505,0.92674,6573,0,0,49.568634
1997,2023-03-20 18:00:00,0.92674,0.92686,0.92549,0.92610,6185,0,0,46.994279
1998,2023-03-20 19:00:00,0.92610,0.92840,0.92583,0.92837,3748,0,0,55.768755


In [9]:
df_an = apply_patterns(df_an)

In [10]:
ema = 200
df_an[f'EMA_{ema}'] = df_an.close.ewm(span=ema, min_periods=ema).mean()

In [11]:
our_cols = ['time', 'open', 'close', 'low', 'high',
                'spread', 'ENGULFING', 'direction', 'EMA_200', 'RSI_14' ]

In [12]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [13]:
df_slim.head()

,time,open,close,low,high,spread,ENGULFING,direction,EMA_200,RSI_14
0,2022-12-01 18:00:00,0.93921,0.93773,0.93681,0.93940,0,False,-1,0.947381,33.627151
1,2022-12-01 19:00:00,0.93773,0.93804,0.93729,0.94004,0,False,1,0.947274,34.682727
2,2022-12-01 20:00:00,0.93803,0.93716,0.93716,0.93838,0,True,-1,0.947158,33.074674
3,2022-12-01 21:00:00,0.93718,0.93662,0.93654,0.93776,0,False,-1,0.947037,32.091406
4,2022-12-01 22:00:00,0.93662,0.93696,0.93652,0.93775,0,False,1,0.946922,33.433254


In [14]:
BUY = 1
SELL = -1
NONE = 0
RSI_LIMIT = 50.0

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.low > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY
        if row.direction == SELL and row.high < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE    

In [15]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [16]:
df_slim.head()

,time,open,close,low,high,spread,ENGULFING,direction,EMA_200,RSI_14,SIGNAL
0,2022-12-01 18:00:00,0.93921,0.93773,0.93681,0.93940,0,False,-1,0.947381,33.627151,0
1,2022-12-01 19:00:00,0.93773,0.93804,0.93729,0.94004,0,False,1,0.947274,34.682727,0
2,2022-12-01 20:00:00,0.93803,0.93716,0.93716,0.93838,0,True,-1,0.947158,33.074674,-1
3,2022-12-01 21:00:00,0.93718,0.93662,0.93654,0.93776,0,False,-1,0.947037,32.091406,0
4,2022-12-01 22:00:00,0.93662,0.93696,0.93652,0.93775,0,False,1,0.946922,33.433254,0


In [17]:
df_slim["SIGNAL"].value_counts()

 0    1649
-1      86
 1      65
Name: SIGNAL, dtype: int64

In [18]:
LOSS_FACTOR = -1.0
PROFIT_FACTOR = 1.5
# consider spread !! 
    
def apply_take_profit(row):
    if row.SIGNAL == BUY:
        return (row.close - row.open) * 1.5 + row.close + SPREAD
    elif row.SIGNAL == SELL:
        return (row.close - row.open) * 1.5 + row.close 
    else:
        return 0.0

def apply_stop_loss(row):
    if row.SIGNAL == BUY:
        return row.open + SPREAD
    elif row.SIGNAL == SELL:
        return row.open
    else:
        return 0.0
    

In [19]:
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)


In [20]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,open,close,low,high,spread,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,SL,TP
377,2022-12-23 11:00:00,0.93064,0.93126,0.93033,0.93142,0,True,1,0.929863,59.598929,1,0.93074,0.932290
386,2022-12-23 20:00:00,0.93290,0.93377,0.93282,0.93426,0,True,1,0.930043,65.115580,1,0.93300,0.935175
390,2022-12-27 00:00:00,0.93117,0.93162,0.93083,0.93236,24,True,1,0.930157,50.493388,1,0.93127,0.932395
503,2023-01-03 17:00:00,0.93293,0.93821,0.93276,0.93827,0,True,1,0.928054,67.310186,1,0.93303,0.946230
510,2023-01-04 00:00:00,0.93574,0.93605,0.93561,0.93640,8,True,1,0.928603,60.175442,1,0.93584,0.936615


In [21]:
df_slim[df_slim.SIGNAL==SELL].head()

,time,open,close,low,high,spread,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,SL,TP
2,2022-12-01 20:00:00,0.93803,0.93716,0.93716,0.93838,0,True,-1,0.947158,33.074674,-1,0.93803,0.935855
11,2022-12-02 05:00:00,0.93822,0.93751,0.93732,0.93842,0,True,-1,0.946186,38.724532,-1,0.93822,0.936445
14,2022-12-02 08:00:00,0.93777,0.93700,0.93648,0.93779,0,True,-1,0.945893,37.392013,-1,0.93777,0.935845
18,2022-12-02 12:00:00,0.93512,0.93370,0.93346,0.93570,0,True,-1,0.945400,28.665581,-1,0.93512,0.931570
27,2022-12-02 21:00:00,0.93836,0.93739,0.93733,0.93934,0,True,-1,0.944677,47.493900,-1,0.93836,0.935935


In [53]:
df_plot = df_slim#.iloc[-100:]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]

markers = ['close', 'TP', 'SL']
marker_colors = ['#0000FF', '#00FF00', '#FF0000']

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = 'markers',
        marker=dict(color=marker_colors[i], size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=['RSI_14'], height=600,width=1600)

In [23]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name
        self.start_price = row.close
        self.trigger_price = row.close
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 0
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.high >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.high)
            elif row.low <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.low)
        if self.SIGNAL == SELL:
            if row.low + SPREAD <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.low + SPREAD)
            elif row.high + SPREAD >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.high + SPREAD)   


# class Trade:
#     def __init__(self, row):
#         self.running = True
#         self.start_index = row.name
#         self.start_price = row.close
#         self.trigger_price = row.close
#         self.SIGNAL = row.SIGNAL
#         self.TP = row.TP
#         self.SL = row.SL
#         self.result = 0.0
#         self.end_time = row.time
#         self.start_time = row.time
#         self.duration = 0
        
#     def close_trade(self, row, result, trigger_price):
#         self.running = False
#         self.result = result
#         self.end_time = row.time
#         self.trigger_price = trigger_price
        
#     def update(self, row):
#         self.duration += 1
#         if self.SIGNAL == BUY:
#             if row.high >= self.TP:
#                 self.close_trade(row, PROFIT_FACTOR, row.high)
#             elif row.low <= self.SL:
#                 self.close_trade(row, LOSS_FACTOR, row.low)
#         if self.SIGNAL == SELL:
#             if row.low <= self.TP:
#                 self.close_trade(row, PROFIT_FACTOR, row.low)
#             elif row.high >= self.SL:
#                 self.close_trade(row, LOSS_FACTOR, row.high)  

In [24]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]
    
    if row.SIGNAL != NONE:
        open_trades.append(Trade(row))   

In [25]:
closed_trades[0].result

-1.0

In [26]:
df_results = pd.DataFrame.from_dict([vars(x) for x in closed_trades])
df_results.head()

,running,start_index,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2,0.93716,0.93839,-1,0.935855,0.93803,-1.0,2022-12-02 02:00:00,2022-12-01 20:00:00,6
1,False,11,0.93751,0.93483,-1,0.936445,0.93822,1.5,2022-12-02 09:00:00,2022-12-02 05:00:00,4
2,False,14,0.93700,0.93483,-1,0.935845,0.93777,1.5,2022-12-02 09:00:00,2022-12-02 08:00:00,1
3,False,18,0.93370,0.94406,-1,0.931570,0.93512,-1.0,2022-12-02 15:00:00,2022-12-02 12:00:00,3
4,False,27,0.93739,0.93863,-1,0.935935,0.93836,-1.0,2022-12-05 00:00:00,2022-12-02 21:00:00,3


In [27]:
df_results.result.value_counts()

-1.0    89
 1.5    62
Name: result, dtype: int64

In [28]:
df_results.head()

,running,start_index,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2,0.93716,0.93839,-1,0.935855,0.93803,-1.0,2022-12-02 02:00:00,2022-12-01 20:00:00,6
1,False,11,0.93751,0.93483,-1,0.936445,0.93822,1.5,2022-12-02 09:00:00,2022-12-02 05:00:00,4
2,False,14,0.93700,0.93483,-1,0.935845,0.93777,1.5,2022-12-02 09:00:00,2022-12-02 08:00:00,1
3,False,18,0.93370,0.94406,-1,0.931570,0.93512,-1.0,2022-12-02 15:00:00,2022-12-02 12:00:00,3
4,False,27,0.93739,0.93863,-1,0.935935,0.93836,-1.0,2022-12-05 00:00:00,2022-12-02 21:00:00,3


In [29]:
def calc_bernuli(nb_tp,nb_sl,balance):
    loss_margin = 0.1
    rivard_to_risk = 1.5
    return balance*( (1+rivard_to_risk*loss_margin)**nb_tp )*( (1-loss_margin)**(nb_sl) )

In [30]:
def calc_bernuli2(nb_tp,nb_sl,balance,profit_to_loss):
    loss_margin = 0.03
    return balance*( (1+profit_to_loss*loss_margin)**nb_tp )*( (1-loss_margin)**(nb_sl) )

In [31]:
df_results['balance'] = 0
df_results['balance'][0] = 100

C:\Users\meysam\AppData\Local\Temp\ipykernel_7192\768337798.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
for i in range(1,len(df_results)):
    if(df_results.result[i] == -1):
        df_results['balance'][i] = calc_bernuli2(0,1,df_results['balance'][i-1],0)
    elif(df_results.result[i] == 1.5):
        profit_to_loss = abs( (df_results['start_price'][i]-df_results['TP'][i])/((df_results['start_price'][i]-df_results['SL'][i])) )
        print(profit_to_loss)
        df_results['balance'][i] = calc_bernuli2(1,0,df_results['balance'][i-1],profit_to_loss)

1.5000000000000782
1.5
1.5
1.4999999999999611
1.4999999999999207
1.5
1.5
1.5
1.5
1.5000000000000508
1.5
2.214285714285556
1.5000000000004625
1.5000000000001734
1.5
2.6904761904758128
2.4259259259258497
1.7631578947368114
1.5
1.6798561151078935
1.9098360655737256
1.6116071428571295
1.5
1.4999999999999467
1.5000000000001068
1.4999999999999725
1.5
1.5
1.5
1.5
1.5
1.5
1.9545454545453629
1.7403846153845262
1.5000000000000533
1.4999999999999132
1.4999999999995375
1.7777777777778052
2.1097560975608767
1.5
1.5000000000000804
1.7747252747252547
1.9098360655737256
1.7314814814814625
1.6302083333333457
1.5877192982456267
1.7717391304347432
1.9310344827585546
1.630890052356038
1.7604166666666186
2.0813953488372183
1.5
1.499999999999927
1.499999999999821
2.068181818181761
1.5
1.5
1.499999999999956
1.500000000000012
1.5
1.7032520325202922
1.499999999999854


C:\Users\meysam\AppData\Local\Temp\ipykernel_7192\4144157101.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
df_results['time'] = df_results['end_time']
cp = CandlePlot(df_results, candles=False)
cp.show_plot(line_traces=['balance'])

In [34]:
from_date = datetime.now()
from_date

datetime.datetime(2023, 3, 20, 23, 8, 46, 566249)

In [35]:
rates = Rates(pair, 13*NB_H1_CANDLES, mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5

,time,open,high,low,close,tick_volume,spread,real_volume
0,2022-11-10 01:35:00,0.98388,0.98396,0.98374,0.98393,261,3,0
1,2022-11-10 01:40:00,0.98395,0.98406,0.98384,0.98384,68,2,0
2,2022-11-10 01:45:00,0.98384,0.98388,0.98365,0.98370,57,3,0
3,2022-11-10 01:50:00,0.98371,0.98374,0.98365,0.98370,50,5,0
4,2022-11-10 01:55:00,0.98372,0.98372,0.98351,0.98358,40,0,0
...,...,...,...,...,...,...,...,...
25995,2023-03-20 19:15:00,0.92591,0.92632,0.92583,0.92631,433,0,0
25996,2023-03-20 19:20:00,0.92631,0.92678,0.92626,0.92669,425,0,0
25997,2023-03-20 19:25:00,0.92669,0.92713,0.92667,0.92700,576,0,0
25998,2023-03-20 19:30:00,0.92701,0.92781,0.92694,0.92770,298,0,0


In [36]:
df_m5_slim = df_m5[['time','high', 'low' ]].copy()

In [37]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy() 
len(df_signals)

151

In [38]:
df_signals['m5_start'] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [39]:
df_signals['start_index_h1'] = df_signals.index

In [40]:
df_signals.columns

Index(['time', 'open', 'close', 'low', 'high', 'spread', 'ENGULFING',
       'direction', 'EMA_200', 'RSI_14', 'SIGNAL', 'SL', 'TP', 'm5_start',
       'start_index_h1'],
      dtype='object')

In [41]:
df_signals.drop(['time', 'open', 'high', 'low', 'spread',
                'ENGULFING', 'EMA_200', 'RSI_14', 'direction'], axis=1, inplace=True)

In [42]:
print(len(df_signals))
df_signals.head()

151


,close,SIGNAL,SL,TP,m5_start,start_index_h1
2,0.93716,-1,0.93803,0.935855,2022-12-01 21:00:00,2
11,0.93751,-1,0.93822,0.936445,2022-12-02 06:00:00,11
14,0.93700,-1,0.93777,0.935845,2022-12-02 09:00:00,14
18,0.93370,-1,0.93512,0.931570,2022-12-02 13:00:00,18
27,0.93739,-1,0.93836,0.935935,2022-12-02 22:00:00,27


In [43]:
df_signals.rename(columns={
    'close' : 'start_price',
    'm5_start' : 'time'
}, inplace=True)

In [44]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on='time', how='left')
merged.fillna(0, inplace=True)
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [45]:
print(len(merged))

merged.head()

26000


,time,high,low,start_price,SIGNAL,SL,TP,start_index_h1
0,2022-11-10 01:35:00,0.98396,0.98374,0.0,0,0.0,0.0,0
1,2022-11-10 01:40:00,0.98406,0.98384,0.0,0,0.0,0.0,0
2,2022-11-10 01:45:00,0.98388,0.98365,0.0,0,0.0,0.0,0
3,2022-11-10 01:50:00,0.98374,0.98365,0.0,0,0.0,0.0,0
4,2022-11-10 01:55:00,0.98372,0.98351,0.0,0,0.0,0.0,0


In [46]:

class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1 = row.start_index_h1
        self.start_price = row.start_price
        self.trigger_price = row.start_price
        self.SIGNAL = row.SIGNAL
        self.TP = row.TP
        self.SL = row.SL
        self.result = 0.0
        self.end_time = row.time
        self.start_time = row.time
        self.duration = 1
        
    def close_trade(self, row, result, trigger_price):
        self.running = False
        self.result = result
        self.end_time = row.time
        self.trigger_price = trigger_price
        
    def update(self, row):
        self.duration += 1
        if self.SIGNAL == BUY:
            if row.high >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.high)
            elif row.low <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.low)
        if self.SIGNAL == SELL:
            if row.low + SPREAD <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.low + SPREAD)
            elif row.high + SPREAD >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.high + SPREAD)   

In [47]:
open_trades_m5 = []
closed_trades_m5 = []

for index, row in merged.iterrows():
    if row.SIGNAL != NONE:
        open_trades_m5.append(TradeM5(row))   
        
    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]
    
len(open_trades_m5   )

0

In [48]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [49]:
df_res_m5.result.value_counts()

-1.0    91
 1.5    52
Name: result, dtype: int64

In [50]:
df_res_m5['balance'] = 0
df_res_m5['balance'][0] = 100

for i in range(1,len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        df_res_m5['balance'][i] = calc_bernuli2(0,1,df_res_m5['balance'][i-1],0)
    elif(df_res_m5.result[i] == 1.5):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['TP'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
        print(profit_to_loss)
        df_res_m5['balance'][i] = calc_bernuli2(1,0,df_res_m5['balance'][i-1],profit_to_loss)
        

1.5000000000000782
1.5
1.5
1.4999999999999611
1.5
1.5
1.5
1.5000000000000508
1.5
2.214285714285556
1.5
2.4259259259258497
1.7631578947368114
1.5
1.9098360655737256
1.6798561151078935
1.6116071428571295
1.5
1.4999999999999467
1.4999999999999725
1.5
1.5
1.5
1.7403846153845262
1.5000000000000533
1.4999999999995375
1.4999999999999132
1.7777777777778052
2.1097560975608767
1.5
1.5000000000000804
1.7747252747252547
1.9098360655737256
1.7314814814814625
1.6302083333333457
1.5877192982456267
1.7717391304347432
1.9310344827585546
1.630890052356038
1.7604166666666186
2.0813953488372183
1.5
1.499999999999927
1.499999999999821
2.068181818181761
1.5
1.5
1.499999999999956
1.500000000000012
1.5
1.7032520325202922
1.499999999999854


C:\Users\meysam\AppData\Local\Temp\ipykernel_7192\598112791.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_7192\598112791.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [51]:
df_res_m5['time'] = df_res_m5['end_time']
cp = CandlePlot(df_res_m5, candles=False)
cp.show_plot(line_traces=['balance'])

In [52]:
df_res_m5['balance'].min()

53.366274469603475